In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)

In [2]:
df = pd.read_csv('data/train.csv').drop_duplicates()
df.reset_index(drop=True,inplace=True)
df

,District,Bathroom,Direction,House Status,Verification,Land Area (m2),Land Used Area (m2),Width (m),Alley Width (m),Price (bil),Price/m2 (mil),Floor,Have Mezzanine,Have Rooftop,Have Basement,Have Attic,Near Market,Near Hospital,Near Center,Near Frontage,Securities Space,Comfortable Space,Have Car Park,Larger Rear,Alley Surface,Urgent Sale,Category
0,17,1.000000,3,1,2,3.960813,4.195697,2.091864,1.098612,5.300000,102.900000,2.000000,0.000000,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1
1,17,3.000000,3,2,2,4.070735,5.150977,1.774952,1.163151,11.500000,199.700000,3.000000,0.000000,0.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,1
2,17,2.000000,3,2,2,3.044522,4.110874,1.609438,0.916291,3.500000,175.000000,3.000000,0.000000,0.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1
3,17,4.000000,3,2,2,3.377588,3.683867,1.665818,1.609438,8.500000,300.400000,5.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,2
4,17,3.000000,2,1,2,3.194583,4.007333,1.517323,2.197225,11.500000,491.500000,3.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42253,15,2.544863,4,2,2,3.772844,4.915817,1.648736,1.753092,6.853184,152.655137,3.544863,0.000000,0.772431,0.0,0.0,0.772431,0.772431,0.772431,0.772431,0.772431,0.000000,0.000000,0.0,0.0,0.0,4
42254,2,2.266755,4,2,2,3.332689,4.152767,1.472658,1.945910,1.766689,65.293298,2.266755,0.000000,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.266755,0.0,0.0,0.0,4
42255,5,1.631259,3,2,2,3.176625,3.640032,1.437963,1.280214,2.441184,105.983763,1.631259,0.368741,0.000000,0.0,0.0,0.368741,0.000000,0.368741,0.368741,0.000000,0.368741,0.000000,0.0,0.0,0.0,4
42256,5,2.392256,4,1,2,3.965120,4.486019,1.609438,1.852226,3.580387,69.209932,2.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.392256,0.607744,0.392256,0.000000,0.000000,0.0,0.0,0.0,4


### Split train/test set

In [3]:
test_df = df.sample(n=9258,random_state=44).reset_index(drop=True)
train_df = df.drop(test_df.index).reset_index(drop=True)

In [4]:
print(train_df.shape)
print(test_df.shape)

(33000, 27)
(9258, 27)


### K-fold cross validation

In [5]:
from sklearn.model_selection import KFold

#Create a new column called kfold and fill it with -1
train_df['kfold'] = - 1
#Initiate the kfold class 
kf = KFold(n_splits=3,random_state=3107,shuffle=True)
#Fill the new kfold column
for fold, (train,val) in enumerate(kf.split(train_df)):
    train_df.loc[val,'kfold'] = fold

In [6]:
train_df['kfold'].value_counts()

0    11000
1    11000
2    11000
Name: kfold, dtype: int64

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score,mean_squared_error

In [8]:
train_df

,District,Bathroom,Direction,House Status,Verification,Land Area (m2),Land Used Area (m2),Width (m),Alley Width (m),Price (bil),Price/m2 (mil),Floor,Have Mezzanine,Have Rooftop,Have Basement,Have Attic,Near Market,Near Hospital,Near Center,Near Frontage,Securities Space,Comfortable Space,Have Car Park,Larger Rear,Alley Surface,Urgent Sale,Category,kfold
0,11,3.000000,1,1,2,2.944439,3.931826,1.490654,1.098612,2.700000,150.000000,2.000000,1.000000,0.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1,2
1,11,3.000000,4,1,2,3.799974,4.964242,1.504077,1.406097,7.800000,178.500000,3.000000,0.000000,1.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1,0
2,11,2.000000,3,1,2,3.394508,4.070735,1.927164,1.635106,3.300000,114.600000,2.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.0,0.0,1,2
3,11,7.000000,3,1,2,4.996536,5.966916,2.047693,2.098018,17.000000,115.700000,3.000000,1.000000,1.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1,0
4,11,2.000000,3,1,2,2.674149,3.332205,1.401183,1.386294,1.600000,118.500000,2.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,15,2.544863,4,2,2,3.772844,4.915817,1.648736,1.753092,6.853184,152.655137,3.544863,0.000000,0.772431,0.0,0.0,0.772431,0.772431,0.772431,0.772431,0.772431,0.000000,0.000000,0.0,0.0,0.0,4,2
32996,2,2.266755,4,2,2,3.332689,4.152767,1.472658,1.945910,1.766689,65.293298,2.266755,0.000000,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.266755,0.0,0.0,0.0,4,0
32997,5,1.631259,3,2,2,3.176625,3.640032,1.437963,1.280214,2.441184,105.983763,1.631259,0.368741,0.000000,0.0,0.0,0.368741,0.000000,0.368741,0.368741,0.000000,0.368741,0.000000,0.0,0.0,0.0,4,2
32998,5,2.392256,4,1,2,3.965120,4.486019,1.609438,1.852226,3.580387,69.209932,2.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.392256,0.607744,0.392256,0.000000,0.000000,0.0,0.0,0.0,4,1


### Model building and evaluation

In [9]:
def model_running(model_choose,fold,data):
    #Split dataframe
    df_train = data[data['kfold'] != fold].reset_index(drop=True)
    df_test = data[data['kfold'] == fold].reset_index(drop=True)
    
    #Split X_train,X_test,y_train,y_test
    X_train = df_train.drop(['kfold','Price (bil)','Price/m2 (mil)'],axis=1)
    X_test = df_test.drop(['kfold','Price (bil)','Price/m2 (mil)'],axis=1)

    y_train = df_train['Price (bil)']
    y_test = df_test['Price (bil)']
    
    #Fitting model
    model = model_choose
    model.fit(X_train,y_train)

    #Predictions
    y_pred = model.predict(X_test)

    #Model evaluation
    r2 = r2_score(y_test,y_pred)
    rmse = mean_squared_error(y_test,y_pred,squared=False)
    
    return r2,rmse 

#### Linear regression

In [10]:
lr = LinearRegression()
lr_r2 = []
lr_rmse = []
for fold in range(3):
    r2,rmse = model_running(lr,fold,train_df)
    lr_r2.append(r2)
    lr_rmse.append(rmse)

In [11]:
print(lr_r2)
print(f'Mean r square: {np.mean(lr_r2)}')
print(lr_rmse)
print(f'Mean root square error: {np.mean(lr_rmse)}')

[0.7256361744330846, 0.718289501263002, 0.7176932229172428]
Mean r square: 0.7205396328711098
[2.6015409633699655, 2.57906438971359, 2.5927977280164267]
Mean root square error: 2.591134360366661


#### Random forest regressor

In [12]:
rf = RandomForestRegressor(random_state=3107)
rf_r2 = []
rf_rmse = []
for fold in range(3):
    r2,rmse = model_running(rf,fold,train_df)
    rf_r2.append(r2)
    rf_rmse.append(rmse)

In [13]:
print(rf_r2)
print(f'Mean r square: {np.mean(rf_r2)}')
print(rf_rmse)
print(f'Mean root square error: {np.mean(rf_rmse)}')

[0.9750935074653052, 0.9761837130740468, 0.9724492707759747]
Mean r square: 0.9745754971051088
[0.7838329749671552, 0.7498902304563339, 0.8099804462316451]
Mean root square error: 0.7812345505517113


#### XG boost

In [14]:
xgb = XGBRegressor(random_state=3107)
xgb_r2 = []
xgb_rmse = []
for fold in range(3):
    r2,rmse = model_running(xgb,fold,train_df)
    xgb_r2.append(r2)
    xgb_rmse.append(rmse)

In [15]:
print(xgb_r2)
print(f'Mean r square: {np.mean(xgb_r2)}')
print(xgb_rmse)
print(f'Mean root square error: {np.mean(xgb_rmse)}')

[0.9664563033695451, 0.9656677359844, 0.9643695741027349]
Mean r square: 0.9654978711522267
[0.9096466082876796, 0.9003510814422001, 0.9211248361252901]
Mean root square error: 0.9103741752850566


#### Light GBM

In [16]:
lgbm = LGBMRegressor(random_state=3107)
lgbm_r2 = []
lgbm_rmse = []
for fold in range(3):
    r2,rmse = model_running(lgbm,fold,train_df)
    lgbm_r2.append(r2)
    lgbm_rmse.append(rmse)

In [18]:
print(lgbm_r2)
print(f'Mean r square: {np.mean(lgbm_r2)}')
print(lgbm_rmse)
print(f'Mean root square error: {np.mean(lgbm_rmse)}')

[0.9501526898193905, 0.9497112485342082, 0.9469428799879344]
Mean r square: 0.9489356061138444
[1.1088890829127558, 1.089672762646581, 1.1240349669819942]
Mean root square error: 1.1075322708471103


- In 4 models, the best performance is Random Forest. But I will choose the Light GBM model due to the speed of processing to tuning hyperparameters.

### Model tuning

In [19]:
from sklearn.model_selection import RandomizedSearchCV

X = train_df.drop(['kfold','Price (bil)','Price/m2 (mil)'],axis=1)
y = train_df['Price (bil)']

In [20]:
#Max number of bins that feature values will be bucketed in
max_bin = [300,400,500]
#Shrinkage rate
learning_rate = [0.001,0.005,0.01,0.05]
#Number of iterations
num_iterations = [200,500,700,1000,1500,2000]
#Max number of leaves in one tree
num_leaves = [30,50,70,100]
#Boosting type
boosting_type = ['gbdt','dart']
# Create the random grid
random_grid = {'max_bin': max_bin,
               'learning_rate': learning_rate,
               'num_iterations': num_iterations,
               'num_leaves': num_leaves,
               'boosting_type': boosting_type
               }
print(random_grid)

{'max_bin': [300, 400, 500], 'learning_rate': [0.001, 0.005, 0.01, 0.05], 'num_iterations': [200, 500, 700, 1000, 1500, 2000], 'num_leaves': [30, 50, 70, 100], 'boosting_type': ['gbdt', 'dart']}


In [21]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
lgbm = LGBMRegressor(random_state=3107)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lgbm_random = RandomizedSearchCV(estimator = lgbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=3107, n_jobs = -1)
# Fit the random search model
lgbm_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=LGBMRegressor(random_state=3107), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'boosting_type': ['gbdt', 'dart'],
                                        'learning_rate': [0.001, 0.005, 0.01,
                                                          0.05],
                                        'max_bin': [300, 400, 500],
                                        'num_iterations': [200, 500, 700, 1000,
                                                           1500, 2000],
                                        'num_leaves': [30, 50, 70, 100]},
                   random_state=3107, verbose=2)

In [22]:
lgbm_random.best_params_

{'num_leaves': 30,
 'num_iterations': 2000,
 'max_bin': 500,
 'learning_rate': 0.05,
 'boosting_type': 'dart'}

In [26]:
#Fit tuned model
lgbm = LGBMRegressor(random_state=3107,num_leaves=30,num_iterations=2000,max_bin=500,learning_rate=0.05,boosting_type='dart')
lgbm_r2 = []
lgbm_rmse = []
for fold in range(3):
    r2,rmse = model_running(lgbm,fold,train_df)
    lgbm_r2.append(r2)
    lgbm_rmse.append(rmse)

print(lgbm_r2)
print(f'Mean r square: {np.mean(lgbm_r2)}')
print(lgbm_rmse)
print(f'Mean root square error: {np.mean(lgbm_rmse)}')

[0.9725039792546691, 0.9726847504735819, 0.9721463142619764]
Mean r square: 0.9724450146634092
[0.8235731267481914, 0.8030882186737046, 0.8144216706510427]
Mean root square error: 0.8136943386909795


### Final model and evaluation

In [28]:
final_model = LGBMRegressor(random_state=3107,num_leaves=30,num_iterations=2000,max_bin=500,learning_rate=0.05,boosting_type='dart')
final_model.fit(X, y)

import joblib
joblib.dump(final_model,'final_model.sav')

['final_model.sav']

In [29]:
#Final evaluation
final_X_test = test_df.drop(['Price (bil)','Price/m2 (mil)'],axis=1)
final_y_test = test_df['Price (bil)']
pred = final_model.predict(final_X_test)
print('r2:',r2_score(final_y_test,pred))
print('rmse:',mean_squared_error(final_y_test,pred,squared=False))

r2: 0.9571830133393139
rmse: 0.9609154004436522


## For reference: Tuning the Random Forest model

- Noted: This process will make the very long times

In [ ]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor(random_state=3107)
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=3107, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X, y)

# #Bootstrap 
# bootstrap = [True, False]
# #The maximum depth of the tree
# max_depth = [10, 30, 50, 100, None]
# #The number of features to consider when looking for the best split
# max_features = ['auto', 'sqrt']
# #The minimum number of samples required to be at a leaf node
# min_samples_leaf = [1, 2, 4]
# #The minimum number of samples required to split an internal node
# min_samples_split = [2, 5, 10]
# #Number of trees in random forest
# n_estimators = [200, 500, 1000, 1500, 2000]

# # Create the random grid
# random_grid = {'bootstrap': bootstrap,
#                'max_depth': max_depth,
#                'max_features': max_features,
#                'min_samples_leaf': min_samples_leaf,
#                'min_samples_split': min_samples_split,
#                'n_estimators':n_estimators
#                }
# print(random_grid)

# rf_random.best_params_ 